In [1]:
import numpy as np
from sympy import *

In [2]:
class Link:
    def __init__(self, theta=0.0, a=0.0, d=0.0, alpha=.0):
        self.angle, self.theta, self.a, self.d, self.alpha = 0, theta, a, d, alpha

    def calc_trans(self):
        x_rot, x_trans, z_trans, z_rot = np.eye(4), np.eye(4), np.eye(4), np.eye(4)
        x_rot[:-1, :-1] = R.from_euler("xyz", [self.alpha, 0, 0]).as_matrix()
        x_trans[0, -1] = self.a
        z_rot[:-1, :-1] = R.from_euler("xyz", [0, 0, self.theta + self.angle]).as_matrix()
        z_trans[-2, -1] = self.d

        return z_rot @ z_trans @ x_trans @ x_rot
class Robot:
    def __init__(self, link1, link2, link3, link4):
        self.link1, self.link2, self.link3, self.link4 = link1, link2, link3, link4

    def update_effector_estimate(self):
        link1_mat = self.link1.calc_trans()
        link2_mat = self.link2.calc_trans()
        link3_mat = self.link3.calc_trans()
        link4_mat = self.link4.calc_trans()
        joint_to_pos = (link1_mat @ link2_mat @ link3_mat @ link4_mat)[:-1, -1]
        return joint_to_pos

link1 = Link(theta=pi/2, d=2.5, alpha=pi/2)
link2 = Link(theta=pi/2, alpha=pi/2)
link3 = Link(a=3.5, alpha=-pi/2)
link4 = Link(a=3)
robot = Robot(link1, link2, link3, link4)

In [3]:
def calc_trans(theta, d=0.0, a=0.0, alpha=0.0):
    x_rot, x_trans, z_trans, z_rot = eye(4), eye(4), eye(4), eye(4)
    x_rot[:-1, :-1] = Matrix([[1, 0, 0],
                                                    [0, cos(alpha), -sin(alpha)],
                                                    [0, sin(alpha), -cos(alpha)]])
    x_trans[0, -1] = a
    z_rot[:-1, :-1] = Matrix([[cos(theta), -sin(theta), 0],
                                                    [sin(theta), cos(theta), 0],
                                                    [0, 0, 1]])
    z_trans[-2, -1] = d
    return z_rot@z_trans@x_trans@x_rot

In [4]:
t1, t2, t3, t4 = symbols("theta_1"), symbols("theta_2"), symbols("theta_3"), symbols("theta_4")
link1, link2, link3, link4 = robot.link1, robot.link2, robot.link3, robot.link4
link1_mat = calc_trans(t1+link1.theta, d=link2.d, alpha=link1.alpha)
link2_mat = calc_trans(t2+link2.theta, alpha=link2.alpha)
link3_mat = calc_trans(t3, a=link3.a, alpha=-link3.alpha)
link4_mat = calc_trans(t4, a=link4.a)
fk = (link1_mat@link2_mat@link3_mat@link4_mat)[:3, -1]
jac = fk.jacobian([t1, t2, t3, t4])

In [5]:
full_fk = (link1_mat@link2_mat@link3_mat@link4_mat)
with open("forward_rotation.tex", "w") as f:
    f.write(latex(full_fk[:-1, :-1], mode="equation", mat_delim="(", mat_str="array"))


In [6]:
x_y_z = Matrix([symbols("x_e"), symbols("y_e"), symbols("z_e")])

In [7]:
symbols("theta_1")

theta_1

In [8]:
eq_xyz = Eq(x_y_z, full_fk[:-1, -1])
for i,sym in enumerate((t1, t2, t3, t4)):
    for sub, func in zip(("s", "c"), (sin, cos)):
        idx = i+1
        eq_xyz = eq_xyz.replace(func(sym), symbols(f"{sub}_{sym}"))

In [9]:
eq_xyz

Eq(Matrix([
[x_e],
[y_e],
[z_e]]), Matrix([
[ 3.5*c_theta_1*s_theta_3 - 3*c_theta_2*s_theta_1*s_theta_4 + 3.5*c_theta_3*s_theta_1*s_theta_2 + 3*c_theta_4*(c_theta_1*s_theta_3 + c_theta_3*s_theta_1*s_theta_2)],
[3*c_theta_1*c_theta_2*s_theta_4 - 3.5*c_theta_1*c_theta_3*s_theta_2 + 3*c_theta_4*(-c_theta_1*c_theta_3*s_theta_2 + s_theta_1*s_theta_3) + 3.5*s_theta_1*s_theta_3],
[                                                                                 3*c_theta_2*c_theta_3*c_theta_4 + 3.5*c_theta_2*c_theta_3 + 3*s_theta_2*s_theta_4]]))

In [10]:
eq_xyz.replace(sin(t1), symbols("sin_"))

Eq(Matrix([
[x_e],
[y_e],
[z_e]]), Matrix([
[ 3.5*c_theta_1*s_theta_3 - 3*c_theta_2*s_theta_1*s_theta_4 + 3.5*c_theta_3*s_theta_1*s_theta_2 + 3*c_theta_4*(c_theta_1*s_theta_3 + c_theta_3*s_theta_1*s_theta_2)],
[3*c_theta_1*c_theta_2*s_theta_4 - 3.5*c_theta_1*c_theta_3*s_theta_2 + 3*c_theta_4*(-c_theta_1*c_theta_3*s_theta_2 + s_theta_1*s_theta_3) + 3.5*s_theta_1*s_theta_3],
[                                                                                 3*c_theta_2*c_theta_3*c_theta_4 + 3.5*c_theta_2*c_theta_3 + 3*s_theta_2*s_theta_4]]))

In [11]:
preview(eq_xyz, viewer='file', filename='x_y_z.png', dvioptions=['-D','120'])

In [12]:
x, y, z = symbols("x"), symbols("y"), symbols("z")
partial = symbols("\partial")
with evaluate(False):
    left_side = Matrix([[(partial*coord)/(partial*angle) for angle in (t1, t2, t3, t4)] for coord in (x,y,z)])

In [13]:
left_side

Matrix([
[(\partial*x)/((\partial*theta_1)), (\partial*x)/((\partial*theta_2)), (\partial*x)/((\partial*theta_3)), (\partial*x)/((\partial*theta_4))],
[(\partial*y)/((\partial*theta_1)), (\partial*y)/((\partial*theta_2)), (\partial*y)/((\partial*theta_3)), (\partial*y)/((\partial*theta_4))],
[(\partial*z)/((\partial*theta_1)), (\partial*z)/((\partial*theta_2)), (\partial*z)/((\partial*theta_3)), (\partial*z)/((\partial*theta_4))]])

In [14]:
for col_idx in range(4):
    eq_jac = Eq(left_side[:, col_idx], jac[:, col_idx])
    for i,sym in enumerate((t1, t2, t3, t4)):
        for sub, func in zip(("s", "c"), (sin, cos)):
            eq_jac = eq_jac.replace(func(sym), symbols(f"{sub}_{sym}"))
    preview(eq_jac, viewer='file', filename=f'jac_col{col_idx+1}.png', dvioptions=['-D','120'])

In [15]:
eq_jac = Eq(left_side, jac)
eq_jac

Eq(Matrix([
[(\partial*x)/((\partial*theta_1)), (\partial*x)/((\partial*theta_2)), (\partial*x)/((\partial*theta_3)), (\partial*x)/((\partial*theta_4))],
[(\partial*y)/((\partial*theta_1)), (\partial*y)/((\partial*theta_2)), (\partial*y)/((\partial*theta_3)), (\partial*y)/((\partial*theta_4))],
[(\partial*z)/((\partial*theta_1)), (\partial*z)/((\partial*theta_2)), (\partial*z)/((\partial*theta_3)), (\partial*z)/((\partial*theta_4))]]), Matrix([
[(-3*sin(theta_1)*sin(theta_3) + 3*sin(theta_2)*cos(theta_1)*cos(theta_3))*cos(theta_4) - 3.5*sin(theta_1)*sin(theta_3) + 3.5*sin(theta_2)*cos(theta_1)*cos(theta_3) - 3*sin(theta_4)*cos(theta_1)*cos(theta_2),  3*sin(theta_1)*sin(theta_2)*sin(theta_4) + 3*sin(theta_1)*cos(theta_2)*cos(theta_3)*cos(theta_4) + 3.5*sin(theta_1)*cos(theta_2)*cos(theta_3), (-3*sin(theta_1)*sin(theta_2)*sin(theta_3) + 3*cos(theta_1)*cos(theta_3))*cos(theta_4) - 3.5*sin(theta_1)*sin(theta_2)*sin(theta_3) + 3.5*cos(theta_1)*cos(theta_3), -(3*sin(theta_1)*sin(theta_2)*cos

In [120]:
for i,sym in enumerate((t1, t2, t3, t4)):
    for sub, func in zip(("s", "c"), (sin, cos)):
        eq_jac = eq_jac.replace(func(sym), symbols(f"{sub}_{sym}"))

In [62]:
with open("forward_translation.tex", "w") as f:
    f.write(latex(full_fk[:-1, -1], mode="equation", mat_delim="(", mat_str="array"))

In [63]:
with open("forward_full.tex", "w") as f:
    f.write(latex(full_fk, mode="equation", mat_delim="(", mat_str="array"))

In [64]:
with open("jacobian.tex", "w") as f:
    f.write(latex(jac, mode="equation", mat_delim="(", mat_str="array"))

In [53]:
jac

Matrix([
[(-3*sin(theta_1)*sin(theta_3) + 3*sin(theta_2)*cos(theta_1)*cos(theta_3))*cos(theta_4) - 3.5*sin(theta_1)*sin(theta_3) + 3.5*sin(theta_2)*cos(theta_1)*cos(theta_3) - 3*sin(theta_4)*cos(theta_1)*cos(theta_2),  3*sin(theta_1)*sin(theta_2)*sin(theta_4) + 3*sin(theta_1)*cos(theta_2)*cos(theta_3)*cos(theta_4) + 3.5*sin(theta_1)*cos(theta_2)*cos(theta_3), (-3*sin(theta_1)*sin(theta_2)*sin(theta_3) + 3*cos(theta_1)*cos(theta_3))*cos(theta_4) - 3.5*sin(theta_1)*sin(theta_2)*sin(theta_3) + 3.5*cos(theta_1)*cos(theta_3), -(3*sin(theta_1)*sin(theta_2)*cos(theta_3) + 3*sin(theta_3)*cos(theta_1))*sin(theta_4) - 3*sin(theta_1)*cos(theta_2)*cos(theta_4)],
[ (3*sin(theta_1)*sin(theta_2)*cos(theta_3) + 3*sin(theta_3)*cos(theta_1))*cos(theta_4) + 3.5*sin(theta_1)*sin(theta_2)*cos(theta_3) - 3*sin(theta_1)*sin(theta_4)*cos(theta_2) + 3.5*sin(theta_3)*cos(theta_1), -3*sin(theta_2)*sin(theta_4)*cos(theta_1) - 3*cos(theta_1)*cos(theta_2)*cos(theta_3)*cos(theta_4) - 3.5*cos(theta_1)*cos(theta_2)*co